In [1]:
pip install gensim


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Import the libraries 
import nltk 
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
from nltk.stem import WordNetLemmatizer
import numpy as np 
import gensim
from gensim.models import Word2Vec
import pandas as pd 

In [21]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abcd\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [23]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\abcd\AppData\Roaming\nltk_data...


True

In [11]:
df_data = pd.read_csv("C:/Users/abcd/Documents/PM_Manmohan_Speech.csv")

In [16]:
df_data

,Unnamed: 0,Date,Location,Title,Speech
0,0,"May 17, 2014",New Delhi,PM's address to the Nation,"\n\tMy Fellow Citizens,\n\tI address you today..."
1,1,"March 4, 2014","Nay Pyi Taw, Myanmar",PM’s statement at 3rd BIMSTEC Summit,\n\t“I am delighted to return to this beautifu...
2,2,"March 1, 2014",New Delhi,PM’s address at the National Awards Ceremony f...,\n\t“I am very happy to participate in the Nat...
3,3,"February 26, 2014",New Delhi,PM's speech at the foundation laying ceremony ...,\n\t“Today marks a very important step forward...
4,4,"February 21, 2014",New Delhi,PM’s farewell speech to the 15th Lok Sabha,"""Madam Speaker,\n\nAs we come to the end of th..."
...,...,...,...,...,...
1396,1396,"July 31, 2004",Bangkok,Speech by the Prime Minister of India at the i...,"\n\t\t \n\tMr. Chairman,\n\t \n\tExcellencies,..."
1397,1397,"July 26, 2004",New Delhi,PM's speech at the CSIR Society Meeting,"Distinguished Members of the CSIR Society, La..."
1398,1398,"July 3, 2004",New Delhi,"PM's inaugural address at the ""Dialogue on Min...","""It gives me great pleasure to be amongst you..."
1399,1399,"June 29, 2004",New Delhi,PM's inaugural address at the Chief Ministers'...,"""I welcome you all to this Conference which w..."


In [20]:
def cleaned(speech):
    # Sentence Tokenization
    sentence = nltk.sent_tokenize(speech)

    # Creating an empty list to store the cleaned dataset 
    corpus = []

    # Loop through each sentence in the speech
    for i in range(len(sentence)):
        # Remove any non-alphabetic characters in the sentence and convert to lowercase
        review = re.sub("[^a-zA-Z0-9]", " ", sentence[i])
        review = review.lower()
        
        # Tokenize the sentence into a list of words
        review = review.split()
        
        # Lemmatize each word in the sentence and remove stop words
        lemmatizer = WordNetLemmatizer()
        review = [lemmatizer.lemmatize(word) for word in review if word not in set(stopwords.words("english"))]
        
        # Convert the list of words back into a sentence
        review = ' '.join(review)
        
        # Add the cleaned sentence to the corpus
        corpus.append(review)

    return corpus

In [19]:
def vectorize(corpus):

    # Tokenize each sentence in the corpus into a list of words
    corpus_tokenized = [nltk.word_tokenize(sentence) for sentence in corpus]

    # Train the Word2Vec model on the tokenized corpus
    model = Word2Vec(sentences=corpus_tokenized, vector_size=300, window=5, min_count=1, workers=4)

    # Extract the word vectors for each word in the corpus
    vectors = []
    for sentence in corpus_tokenized:
        for word in sentence:
            if word in model.wv.key_to_index:
                # If the word is in the Word2Vec model's vocabulary, extract its vector
                word_vector = model.wv.get_vector(word)
                vectors.append(np.average(word_vector))
            else:
                # If the word is not in the vocabulary, print a message to the console
                print(f"{word} not in vocabulary.") 
    
    return vectors

In [14]:
# To convert cleaned speeches into string for performing NER
def string(speech):
    string_speech = " ".join(speech)
    return string_speech

In [24]:
df_data['Cleaned_Speech'] = df_data['Speech'].apply(cleaned)


In [26]:
df_data['Cleaned_Speech_string'] = df_data['Cleaned_Speech'].apply(string)

In [30]:
df_data.to_csv(r"C:\Users\abcd\Documents\PM_Manmohan_SpeechCleaned.csv")